In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def search_amazon(product_name):

    driver_path = 'path_to_chromedriver'
    driver = webdriver.Chrome(executable_path=driver_path)


    driver.get("https://www.amazon.in/")


    search_box = driver.find_element_by_name("field-keywords")


    search_box.clear()

    search_box.send_keys(product_name)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(30)

    search_results = driver.find_elements_by_css_selector(".s-result-item")

    for result in search_results:
        product_title = result.find_element_by_css_selector(".a-text-normal").text
        product_link = result.find_element_by_css_selector(".a-link-normal").get_attribute("href")
        print(f"Product: {product_title}\nLink: {product_link}\n")

    driver.quit()

if __name__ == "__main__":
    product_name = input("Enter the product you want to search for on Amazon.in: ")
    search_amazon(product_name)


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

def scrape_page(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    products = []

    for result in soup.find_all('div', class_='s-result-item'):
        product_data = {}

        try:
            product_data['Brand Name'] = result.find('span', class_='a-size-base-plus a-color-base').text.strip()
        except AttributeError:
            product_data['Brand Name'] = '-'

        try:
            product_data['Name of the Product'] = result.find('span', class_='a-text-normal').text.strip()
        except AttributeError:
            product_data['Name of the Product'] = '-'

        try:
            product_data['Price'] = result.find('span', class_='a-price-whole').text.strip()
        except AttributeError:
            product_data['Price'] = '-'

        try:
            product_data['Return/Exchange'] = result.find('div', class_='a-row a-size-small').text.strip()
        except AttributeError:
            product_data['Return/Exchange'] = '-'

        try:
            product_data['Expected Delivery'] = result.find('span', class_='a-text-bold').text.strip()
        except AttributeError:
            product_data['Expected Delivery'] = '-'

        try:
            product_data['Availability'] = result.find('span', class_='a-size-medium a-color-price').text.strip()
        except AttributeError:
            product_data['Availability'] = '-'

        try:
            product_data['Product URL'] = result.find('a', class_='a-link-normal')['href']
        except (AttributeError, KeyError):
            product_data['Product URL'] = '-'

        products.append(product_data)

    return products

def scrape_amazon(search_query, num_pages=3):
    driver_path = 'path_to_chromedriver'
    driver = webdriver.Chrome(executable_path=driver_path)
    products = []

    for page_num in range(1, num_pages + 1):
        url = f'https://www.amazon.in/s?k={search_query}&page={page_num}'
        driver.get(url)
        time.sleep(5)  # Allow time for the page to load

        page_source = driver.page_source
        page_products = scrape_page(page_source)
        products.extend(page_products)

    driver.quit()

    df = pd.DataFrame(products)

    df.to_csv('amazon_products.csv', index=False)

if __name__ == "__main__":
    search_query = input("Enter the product you want to search for on Amazon.in: ")
    scrape_amazon(search_query)


In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import os

def scrape_images(keyword, num_images=10):
    driver_path = 'path_to_chromedriver'  # Replace with the actual path to your Chrome WebDriver executable
    download_path = 'image_data/' + keyword

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # Run Chrome in headless mode (without GUI)
    driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

    os.makedirs(download_path, exist_ok=True)

    driver.get("https://www.google.com/imghp")

    search_box = driver.find_element_by_name("q")

    search_box.clear()

    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)

    num_scrolls = num_images // 20
    for _ in range(num_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    img_tags = soup.find_all('img', class_='rg_i')
    img_urls = [img['src'] for img in img_tags if 'src' in img.attrs]

    for i, img_url in enumerate(img_urls[:num_images]):
        img_name = f"{keyword}_{i+1}.jpg"
        img_path = os.path.join(download_path, img_name)
        with open(img_path, 'wb') as img_file:
            img_data = requests.get(img_url).content
            img_file.write(img_data)

    driver.quit()

if __name__ == "__main__":
    keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
    for keyword in keywords:
        print(f"Scraping images for '{keyword}'...")
        scrape_images(keyword)
        print(f"Downloaded {keyword} images successfully.")


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

def scrape_page(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    smartphones = []

    for result in soup.find_all('div', class_='_1AtVbE'):
        smartphone_data = {}

        try:
            smartphone_data['Brand Name'] = result.find('div', class_='_4rR01T').text.strip()
        except AttributeError:
            smartphone_data['Brand Name'] = '-'

        try:
            smartphone_data['Smartphone Name'] = result.find('a', class_='IRpwTa').text.strip()
        except AttributeError:
            smartphone_data['Smartphone Name'] = '-'

        try:
            smartphone_data['Colour'] = result.find('a', class_='IRpwTa').text.strip().split()[-1]
        except AttributeError:
            smartphone_data['Colour'] = '-'

        details = result.find_all('li', class_="rgWa7D")
        for detail in details:
            text = detail.text.strip()
            if "RAM" in text:
                smartphone_data['RAM'] = text.split('|')[0].strip()
            elif "ROM" in text:
                smartphone_data['Storage(ROM)'] = text.split('|')[0].strip()
            elif "Primary Camera" in text:
                smartphone_data['Primary Camera'] = text.split('|')[0].strip()
            elif "Secondary Camera" in text:
                smartphone_data['Secondary Camera'] = text.split('|')[0].strip()
            elif "Display Size" in text:
                smartphone_data['Display Size'] = text.split('|')[0].strip()
            elif "Battery Capacity" in text:
                smartphone_data['Battery Capacity'] = text.split('|')[0].strip()

        try:
            smartphone_data['Price'] = result.find('div', class_='_30jeq3').text.strip()
        except AttributeError:
            smartphone_data['Price'] = '-'

        try:
            product_url = result.find('a', class_='IRpwTa')['href']
            smartphone_data['Product URL'] = 'https://www.flipkart.com' + product_url
        except (AttributeError, KeyError):
            smartphone_data['Product URL'] = '-'

        smartphones.append(smartphone_data)

    return smartphones

def scrape_flipkart(search_query):
    driver_path = 'path_to_chromedriver'  
    driver = webdriver.Chrome(executable_path=driver_path)
    smartphones = []

    url = f'https://www.flipkart.com/search?q={


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

def scrape_coordinates(city):
 
    driver_path = 'path_to_chromedriver'  # Replace with the actual path to your Chrome WebDriver executable
    driver = webdriver.Chrome(executable_path=driver_path)

    try:
        
        driver.get("https://www.google.com/maps")

        search_box = driver.find_element_by_id("searchboxinput")

      
        search_box.clear()

      
        search_box.send_keys(city)
        search_box.send_keys(Keys.RETURN)

        time.sleep(5)

       
        url = driver.current_url
        coordinates = extract_coordinates_from_url(url)

        return coordinates

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
      
        driver.quit()


def extract_coordinates_from_url(url):
    try:
        lat_start = url.index("@") + 1
        lat_end = url.index(",", lat_start)
        lon_start = lat_end + 1
        lon_end = url.index(",", lon_start)

        latitude = float(url[lat_start:lat_end])
        longitude = float(url[lon_start:lon_end])

        return {"Latitude": latitude, "Longitude": longitude}
    except ValueError:
        return None

if __name__ == "__main__":
    city = input("Enter the city name to get its geospatial coordinates: ")
    coordinates = scrape_coordinates(city)
    if coordinates:
        print(f"Geospatial Coordinates for {city}:")
        print(f"Latitude: {coordinates['Latitude']}")
        print(f"Longitude: {coordinates['Longitude']}")
    else:
        print("Coordinates not found.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_digit_in_gaming_laptops():

    url = 'https://www.digit.in/top-products/best-gaming-laptops-40.html'

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        laptop_container = soup.find('div', class_='TopNumList')

        laptop_names = []
        laptop_specs = []

        for laptop_entry in laptop_container.find_all('div', class_='Top-Rated-Slider'):
        
            name = laptop_entry.find('div', class_='TopNumbeTitle').text.strip()
            laptop_names.append(name)

            specs = []
            for spec in laptop_entry.find_all('div', class_='Specs-Wrap'):
                spec_name = spec.find('div', class_='Specs').text.strip()
                spec_value = spec.find('div', class_='value').text.strip()
                specs.append(f'{spec_name}: {spec_value}')
            laptop_specs.append(', '.join(specs))

        df = pd.DataFrame({'Laptop Name': laptop_names, 'Specifications': laptop_specs})

      
        df.to_csv('digit_in_gaming_laptops.csv', index=False)

        print("Scraping completed. Data saved to 'digit_in_gaming_laptops.csv'.")

    else:
        print("Failed to retrieve the web page. Check your internet connection or the URL.")

if __name__ == "__main__":
    scrape_digit_in_gaming_laptops()


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def scrape_forbes_billionaires():

    url = 'https://www.forbes.com/billionaires/'

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        table = soup.find('table', class_='table')

        ranks = []
        names = []
        net_worths = []
        ages = []
        citizenships = []
        sources = []
        industries = []

        for row in table.find_all('tr')[1:]:
            columns = row.find_all('td')

            rank = columns[0].text.strip()
            name = columns[1].text.strip()
            net_worth = columns[2].text.strip()
            age = columns[3].text.strip()
            citizenship = columns[4].text.strip()
            source = columns[5].text.strip()
            industry = columns[6].text.strip()

            ranks.append(rank)
            names.append(name)
            net_worths.append(net_worth)
            ages.append(age)
            citizenships.append(citizenship)
            sources.append(source)
            industries.append(industry)

        df = pd.DataFrame({
            'Rank': ranks,
            'Name': names,
            'Net Worth': net_worths,
            'Age': ages,
            'Citizenship': citizenships,
            'Source': sources,
            'Industry': industries
        })

    
        df.to_csv('forbes_billionaires.csv', index=False)

        print("Scraping completed. Data saved to 'forbes_billionaires.csv'.")

    else:
        print("Failed to retrieve the web page. Check your internet connection or the URL.")

if __name__ == "__main__":
    scrape_forbes_billionaires()


In [ ]:
import os
import googleapiclient.discovery
from google.oauth2 import service_account

credentials_file = 'your_credentials.json'  # Replace with the path to your JSON credentials file

api_service_name = 'youtube'
api_version = 'v3'
scopes = ['https://www.googleapis.com/auth/youtube.force-ssl']

credentials = service_account.Credentials.from_service_account_file(
    credentials_file, scopes=scopes)

youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)

def get_comments(video_id, max_comments=500):
    comments = []

    request = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=max_comments,
        textFormat='plainText'
    )

    while request:
        response = request.execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']
            comment_text = comment['snippet']['textDisplay']
            comment_upvotes = comment['snippet']['likeCount']
            comment_time = comment['snippet']['publishedAt']

            comments.append({
                'Comment': comment_text,
                'Upvotes': comment_upvotes,.,ko
                'Time Posted': comment_time
            })

        request = youtube.commentThreads().list_next(request, response)

    return comments

if __name__ == "__main__":
    video_id = input("Enter the YouTube video ID (the part after 'v='): ")
    comments = get_comments(video_id)

    if comments:
        print(f"Total Comments: {len(comments)}")
        for i, comment in enumerate(comments):
            print(f"Comment {i + 1}:")
            print(f"Comment: {comment['Comment']}")
            print(f"Upvotes: {comment['Upvotes']}")
            print(f"Time Posted: {comment['Time Posted']}")
            print()
    else:
        print("No comments found for the given video.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def scrape_hostels_in_london():
    url = 'https://www.hostelworld.com/s?q=London&country=England'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        hostel_list = []


        hostel_elements = soup.find_all('div', class_='fabresult rounded clearfix')

        for hostel_element in hostel_elements:
            hostel_data = {}

            hostel_data['Hostel Name'] = hostel_element.find('h2', class_='title-3').text.strip()
            hostel_data['Distance from City Centre'] = hostel_element.find('span', class_='description').text.strip()
            hostel_data['Ratings'] = hostel_element.find('div', class_='score orange').text.strip()
            hostel_data['Total Reviews'] = hostel_element.find('div', class_='reviews')
            hostel_data['Overall Reviews'] = hostel_data['Total Reviews'].find('a').text.strip()
            hostel_data['Privates From Price'] = hostel_element.find('span', class_='price').text.strip()
            hostel_data['Dorms From Price'] = hostel_element.find('div', class_='prices').find_all('span')[1].text.strip()
            hostel_data['Facilities'] = ', '.join([facility.text.strip() for facility in hostel_element.find_all('div', class_='facilities')])

            hostel_data['URL'] = 'https://www.hostelworld.com' + hostel_element.find('a', class_='title-2')['href']

            property_url = hostel_data['URL']
            property_response = requests.get(property_url)
            if property_response.status_code == 200:
                property_soup = BeautifulSoup(property_response.text, 'html.parser')
                property_description = property_soup.find('div', class_='content-style-3').text.strip()
                hostel_data['Property Description'] = property_description
            else:
                hostel_data['Property Description'] = '-'

            hostel_list.append(hostel_data)

        df = pd.DataFrame(hostel_list)

        df.to_csv('hostelworld_london_hostels.csv', index=False)

        print("Scraping completed. Data saved to 'hostelworld_london_hostels.csv'.")

    else:
        print("Failed to retrieve the web page. Check your internet connection or the URL.")

if __name__ == "__main__":
    scrape_hostels_in_london()
